# ĐỒ ÁN CUỐI KỲ: DỰ ĐOÁN GIÁ ĐIỆN THOẠI

# 1. Thu Thập dữ liệu

In [1]:
# Thư viện cần thiết
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import re
import time
import random
import pandas as pd
import csv
import math
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns # seaborn là thư viện được xây trên matplotlib, giúp việc visualization đỡ khổ hơn
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn import set_config
set_config(display='diagram') # Để trực quan hóa pipeline
import sklearn


Parse trang web:  để lấy link vào trang thông tin chi tiết từng sản phẩm

In [2]:

def collect_phone_links(link):
    # Tương tác với trang web, bấm nút xem thêm cho đến khi hết trang web (khoảng 7 lần bấm)
    browser = webdriver.Chrome(executable_path='./chromedriver.exe') 
    browser.get(link)
    i = 0
    while i < 7:
        browser.find_element_by_class_name('loadmore').click()
        i = i + 1
    html_text = browser.page_source
    browser.quit()
    # parse html để lấy phone links
    phone_link_list = []
    parse_html = BeautifulSoup(html_text, 'html.parser')
    phones = parse_html.find_all('a', {'class': 'prdImageHrf'})
    for phone in phones:
        phone_link = 'https://www.dienmayxanh.com/' + phone['href']
        phone_link_list.append(phone_link)
    return phone_link_list
    

In [3]:
def write_links_into_file(link):
    data_link = open('phone_links.txt','w')
    links =  collect_phone_links(link)
    for link in links:
        data_link.write(link + '\n')
    data_link.close()

In [4]:
#start_link = 'https://www.dienmayxanh.com/dien-thoai'
#write_links_into_file(start_link)

Sau khi đã lấy được tất cả các phone link và lưu vào file phone_links.txt để dùng về sau. Vì vậy ở các lần chạy sau không cần chạy lên phần này nữa -> ta comment dòng: write_links_into_file(link)

In [5]:
phone_links = open('phone_links.txt','r')
raw_links  = phone_links.readlines()
phone_links.close()
links = []
for link in raw_links:
    link = link[:-1]
    links.append(link)

    

In [6]:
print('Tổng số phone link ta có: ',len(links))

Tổng số phone link ta có:  179


Parse mỗi trang web tương ứng với mỗi link để lấy dữ liệu điện thoại

In [7]:
def get_phone_infor(links):
    infor = []
    for link in links:
        get_html = requests.get(link).text
        parse_html = BeautifulSoup(get_html, 'html.parser')
        
        name = re.search(r'(?<=dien-thoai/)[\w-]+', link)
        get_phone_name =  name.group(0)
        print(name)
        
        brand = re.search(r'(?<=dien-thoai/)[\w]+', link)
        get_brand = brand.group(0)
        
        tech_infor = parse_html.find('ul',{'class': 'specs'})
        if tech_infor!= None:
            get_tech_infor = tech_infor.text
        else:
            continue
        get_screen = re.search(r'(([\d.]+))"', get_tech_infor)
        if get_screen!= None:
            screen = get_screen.group(1)
        else:
            screen = np.nan
            
        get_operation = re.search(r'((?<=Hệ điều hành:)[\w ]+)([()\w ])*[A-Z]', get_tech_infor)
        if get_operation!= None:
            operation = get_operation.group(1)
        else:
            operation = np.nan
        
        get_main_camera = re.search(r'((?<=Camera sau:)([\w ]+)*[\d.]+) [MP&]', get_tech_infor)
        if get_main_camera!= None:
            main_camera = get_main_camera.group(1)
        else:
            main_camera = np.nan
            
        get_front_camera = re.search(r'((?<=Camera trước:)([\w ]+)*[\d.]+) MP', get_tech_infor)
        if get_front_camera!= None:
            front_camera = get_front_camera.group(1)
        else:
            front_camera = np.nan
            
        get_CPU =re.search(r'((?<=CPU:)[\w]+)', get_tech_infor)
        if get_CPU!= None:
            CPU = get_CPU.group(1)
        else:
            CPU = np.nan
        get_RAM = re.search(r'((?<=(RAM:))[\d]+) GB[Bộ]*', get_tech_infor)
        if get_RAM!= None:
            RAM = get_RAM.group(1)
        else:
            RAM = np.nan
            
        get_storage = re.search(r'((?<=Bộ nhớ trong:)[\d]+)[ GB]', get_tech_infor)
        if get_storage != None:
            storage = get_storage.group(1)
        else:
            storage = np.nan
        
        get_battery = re.search(r'((?<=Dung lượng pin:)[\w ]+)mAh', get_tech_infor)
        if get_battery!= None:
            battery = get_battery.group(1)
        else:
            battery = np.nan
            
        get_price = parse_html.find('strong',{'class': 'dl'})
        if get_price != None:
            price = get_price.text
        elif (parse_html.find('div',{'class': 'area_price'}) !=None) and (parse_html.find('span',{'class': 'nosell'}) == None) :
            get_price = parse_html.find('div',{'class': 'area_price'}).text
            price = re.search(r'([\d.]+)₫', get_price).group(1)
        else:
            price = np.nan
        print(price)
        
        infor.append([get_phone_name,get_brand,screen,operation,main_camera,front_camera,CPU,RAM,storage,battery,price])
        
        
        time.sleep(1)
    return infor
    
    

In [8]:
#data_phones = get_phone_infor(links)

In [9]:
#df = pd.DataFrame(data = data_phones, columns = ["Phone_Name","Brand","Screen","Operation","Main_Camera","Front_Camera","CPU","RAM","Storage","Battery",'Price'])
#df.to_csv("Phone_Information.csv",index = False)
#df.sample(20)

In [10]:
df = pd.read_csv('Phone_Information.csv')

In [29]:
df.head(20)

,Phone_Name,Brand,Screen,Operation,Main_Camera,Front_Camera,CPU,RAM,Storage,Battery,Price
0,oppo-reno5,oppo,6.43,Android 11,Chính 64,44.0,Snapdragon,8.0,128.0,4310,8690000
1,oppo-a92,oppo,6.50,Android 10,Chính 48,16.0,Snapdragon,8.0,128.0,5000,6490000
2,iphone-12-mini,iphone,5.40,iOS 14,2 camera 12,12.0,Apple,4.0,64.0,2227,21990000
3,xiaomi-mi-10t-pro,xiaomi,6.67,Android 10,Chính 108,20.0,Snapdragon,8.0,256.0,5000,12990000
4,samsung-galaxy-a12-6gb,samsung,6.50,Android 10,Chính 48,8.0,MediaTek,6.0,128.0,5000,4690000
5,samsung-galaxy-m51,samsung,6.70,Android 10,Chính 64,32.0,Snapdragon,8.0,128.0,7000,9490000
6,realme-c15,realme,6.50,Android 10,Chính 13,8.0,Snapdragon,4.0,64.0,6000,4190000
7,vivo-y51-2020,vivo,6.58,Android 11,Chính 48,16.0,Snapdragon,8.0,128.0,5000,6290000
8,samsung-galaxy-z-fold-2,samsung,7.59,Android 10,3 camera 12,10.0,Snapdragon,12.0,256.0,4500,50000000
9,samsung-galaxy-z-fold2-5g-dac-biet,samsung,7.59,Android 10,3 camera 12,10.0,Snapdragon,12.0,256.0,4500,50000000


**Bước tiếp theo:**
_Loại bỏ dòng trùng

_Biến cột Screen,Front Camera, Ram, storage, battery (nhớ bỏ khoảng trắng sau số) về dạng float

_Cột Main_camera bỏ hết ký tự, chừa lại số, nếu có "2 camera 12" thì lấy 2 nhân 12 -> đưa về dạng float

_Cột Price bỏ ký tự khác không phải là số -> đưa về dạng float

_cột Operation chỉ lấy tên bỏ số


# Khám phá dữ liệu


### Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

In [12]:
df.shape


(175, 11)

### Dữ liệu có các dòng bị lặp không?

In [13]:
df.index.duplicated().sum()

0

### Dữ liệu có bị thiếu không ?

In [14]:
df.isna().sum()

Phone_Name       0
Brand            0
Screen           0
Operation       38
Main_Camera     20
Front_Camera    40
CPU             38
RAM             41
Storage         37
Battery          0
Price            1
dtype: int64

# Khám phá dữ liệu (để biết cách tách các tập)

In [15]:
# Cột output hiện có kiểu dữ liệu gì?
df['Price'].dtype

dtype('O')

In [16]:
# Cột output có giá trị thiếu không?
df['Price'].isna().sum()

1

In [17]:
# Tỉ lệ các lớp trong cột output?
df['Price'].value_counts(normalize=True) * 100

 4.990.000₫ -6%       1.724138
200.000               1.724138
23.990.000            1.724138
 2.990.000₫ -3%       1.724138
 990.000₫ -10%        1.149425
                        ...   
 3.990.000₫ -12%      0.574713
 23.990.000₫ -16%     0.574713
 1.190.000₫ -10%      0.574713
5.990.000             0.574713
 39.990.000₫ -2%      0.574713
Name: Price, Length: 142, dtype: float64

- Cột Price có kiểu dữ liệu Objects Chúng ta cần chuyển sang dạng số mới có thể tách được tập
- Cột Price có giá trị thiếu nên ta cần loại bỏ dòng thiếu đi.
- Tỉ lệ các lớp không bị chênh lệch nhau quá nhiều nên không cần làm gì cả

In [18]:
df.drop(df[df['Price'].isna()].index,inplace =True)

In [19]:
df['Price']= df['Price'].str.extract(r'(\d+\.\d+[.]?[\d]*)', expand=False)
df['Price']= df['Price'].str.replace(r'[.]','',regex=True).astype('int64')

In [20]:
df['Price']

0       8690000
1       6490000
2      21990000
3      12990000
4       4690000
         ...   
170      200000
171      190000
172      190000
173      170000
174      160000
Name: Price, Length: 174, dtype: int64

In [26]:
df.Front_Camera = pd.to_numeric(df.Front_Camera.str.extract(r'([\d.]+)', expand=False), errors='coerce')
df.

In [31]:
df.Front_Camera.isna().sum()

40

# Tiền xử lý (tách các tập)


In [ ]:
# Tách X và y
y_pr = df["Price"] 
X_df = df.drop("Price", axis=1)

In [ ]:
# Tách tập huấn luyện và tập test theo tỉ lệ 80%:20%
train_X_df, test_X_df, train_y_pr, test_y_pr = train_test_split(X_df, y_pr, test_size=0.2, 
                                                              random_state=0)


In [ ]:
train_X_df, val_X_df, train_y_pr, val_y_pr = train_test_split(train_X_df, train_y_pr, test_size=0.2, 
                                                               random_state=0)

In [ ]:
train_X_df.shape

In [ ]:
train_y_pr.shape

In [ ]:
test_X_df.shape

In [ ]:
test_y_pr.shape

In [ ]:
val_X_df.shape

In [ ]:
val_y_pr.shape

# Khám phá dữ liệu (tập huấn luyện)

In [ ]:
train_X_df.dtypes

In [ ]:
phones_df.rear_camera = pd.to_numeric(phones_df.rear_camera.str.extract(r'([\d.]+)', expand=False), errors='coerce')